# Face Detection Techniques

<p> Libraries

In [1]:
import cv2

from retinaface import RetinaFace
import matplotlib.pyplot as plt

from ultralytics import YOLO

2024-09-23 20:38:23.888327: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-23 20:38:23.912912: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-23 20:38:23.922126: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-23 20:38:23.946196: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-23 20:38:26.106424: W tensorflow/compiler/tf2

### Haarcascade

#### Manual Exemple

In [2]:
img = cv2.imread('./images/faces_no_boxes_7.jpg') 
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 

haar_cascade = cv2.CascadeClassifier('data/haarcascades/haarcascade_frontalface_alt.xml') 

faces_rect = haar_cascade.detectMultiScale(gray_img, 1.1, 9) 

for (x, y, w, h) in faces_rect: 
	cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2) 

while True:
    cv2.imshow('Detected faces', img) 

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

#### Automated

In [3]:
def detectAndWriteCascade(img_path, annotated_path):
    img = cv2.imread(img_path) 
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    
    haar_cascade = cv2.CascadeClassifier('data/haarcascades/haarcascade_frontalface_alt.xml') 
    
    faces_rect = haar_cascade.detectMultiScale(gray_img, 1.1, 9) 
    
    for (x, y, w, h) in faces_rect: 
    	cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2) 
    
    cv2.imwrite(annotated_path, img)

In [4]:
for i in range(10):
    detectAndWriteCascade(f'./images/faces_no_boxes_{i+1}.jpg', f'./images/faces_cascade_{i+1}.jpg')

### Retinaface

#### Manual Example

In [5]:
image_path = './images/faces_no_boxes_7.jpg'

faces_info = RetinaFace.detect_faces(image_path)
img = cv2.imread(image_path) 

for face_id in faces_info.keys():
    face = faces_info[face_id]
    
    x1, y1, x2, y2 = face["facial_area"]
    cv2.rectangle(img, (x2, y2), (x1, y1), (0, 255, 0), 2)

    score = face['score']
    label = f"Face: {score:.2f}"
    cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

while True:
    cv2.imshow('Detected faces', img) 

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

2024-09-23 20:39:00.200625: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 119275520 exceeds 10% of free system memory.
2024-09-23 20:39:00.372734: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 120004608 exceeds 10% of free system memory.
2024-09-23 20:39:00.516636: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 119275520 exceeds 10% of free system memory.
2024-09-23 20:39:00.739357: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 119275520 exceeds 10% of free system memory.
2024-09-23 20:39:00.860616: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 119275520 exceeds 10% of free system memory.


#### Automated

In [6]:
def detectAndWriteRetinaface(img_path, annotated_path):
    faces_info = RetinaFace.detect_faces(img_path)
    img = cv2.imread(img_path) 
    
    for face_id in faces_info.keys():
        face = faces_info[face_id]
        
        x1, y1, x2, y2 = face["facial_area"]
        cv2.rectangle(img, (x2, y2), (x1, y1), (0, 255, 0), 2)
        
        score = face['score']
        label = f"Face: {score:.2f}"
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imwrite(annotated_path, img)

In [7]:
for i in range(10):
    detectAndWriteRetinaface(f'./images/faces_no_boxes_{i+1}.jpg', f'./images/faces_retinaface_{i+1}.jpg')

### YOLO

#### Manual Example

In [8]:
model = YOLO('yolov8n.pt')  # Choose a YOLOv8 variant

# Load the image or video
image_path = './images/faces_no_boxes_7.jpg'
img = cv2.imread(image_path)

# Perform inference
results = model(image_path)

# Access bounding boxes and confidence scores
for result in results:
    boxes = result.boxes
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        confidence = box.conf[0]
        class_id = box.cls[0]
        label = f"Person: {confidence:.2f}"
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


while True:
    cv2.imshow('Detected faces', img) 

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


image 1/1 /home/stnz/coding/exercises_2024/classes/computer_vision/lab09/images/faces_no_boxes_7.jpg: 384x640 10 persons, 287.0ms
Speed: 5.0ms preprocess, 287.0ms inference, 4.8ms postprocess per image at shape (1, 3, 384, 640)


#### Automated

In [9]:
def detectAndWriteYOLO(img_path, annotated_path):
    model = YOLO('yolov8n.pt') 
    
    img = cv2.imread(img_path)
    
    results = model(img_path)
    
    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
            confidence = box.conf[0]
            class_id = box.cls[0]
            label = f"Person: {confidence:.2f}"
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imwrite(annotated_path, img)

In [10]:
for i in range(10):
    detectAndWriteYOLO(f'./images/faces_no_boxes_{i+1}.jpg', f'./images/faces_yolo_{i+1}.jpg')


image 1/1 /home/stnz/coding/exercises_2024/classes/computer_vision/lab09/images/faces_no_boxes_1.jpg: 320x640 16 persons, 175.3ms
Speed: 3.2ms preprocess, 175.3ms inference, 1.3ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 /home/stnz/coding/exercises_2024/classes/computer_vision/lab09/images/faces_no_boxes_2.jpg: 384x640 5 persons, 108.1ms
Speed: 2.7ms preprocess, 108.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /home/stnz/coding/exercises_2024/classes/computer_vision/lab09/images/faces_no_boxes_3.jpg: 384x640 13 persons, 1 cup, 122.3ms
Speed: 2.7ms preprocess, 122.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /home/stnz/coding/exercises_2024/classes/computer_vision/lab09/images/faces_no_boxes_4.jpg: 384x640 3 persons, 4 teddy bears, 114.7ms
Speed: 2.7ms preprocess, 114.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /home/stnz/coding/exercises_2024/classes/computer_vis